In [13]:
import pickle
import pymongo
import pandas as pd
from bson import ObjectId

In [14]:
with open('et_classifier.pkl', 'rb') as f:
    et_classifier = pickle.load(f)
with open('gb_classifier.pkl', 'rb') as f:
    gb_classifier = pickle.load(f)
with open('hgb_classifier.pkl', 'rb') as f:
    hgb_classifier = pickle.load(f)
with open('rf_classifier.pkl', 'rb') as f:
    rf_classifier = pickle.load(f)
with open('ab_classifier.pkl', 'rb') as f:
    ab_classifier = pickle.load(f)

In [15]:
MONGO_CLIENT = pymongo.MongoClient("mongodb://192.168.1.7:27017/")
SDA_DB = MONGO_CLIENT["steam_dota_api"]
MLRS_COL = SDA_DB["multi_real_time_stats"]

In [16]:
# matches = MLRS_COL.find({"_id": ObjectId("65b5e65abb22a1e961d58c20")})
matches = MLRS_COL.find({"match.match_id": "7557978739"})
matches_for_pd = []

for entry in matches:
    try:
        tmp = {}

        tmp["timestamp"] = entry["match"]["timestamp"]
        tmp["game_time"] = entry["match"]["game_time"]

        tmp['score'] = entry["teams"][0]["score"] - entry["teams"][1]["score"]
        tmp['net_worth'] = entry["teams"][0]["net_worth"] - entry["teams"][1]["net_worth"]

        for team in entry["teams"]:
            for p_idx, player in enumerate(team["players"]):
                # tmp["team_{}_player_{}_heroid".format(team["team_number"], p_idx)] = player['heroid']
                tmp["team_{}_player_{}_level".format(team["team_number"], p_idx)] = player['level']
                tmp["team_{}_player_{}_kill_count".format(team["team_number"], p_idx)] = player['kill_count']
                tmp["team_{}_player_{}_death_count".format(team["team_number"], p_idx)] = player['death_count']
                tmp["team_{}_player_{}_assists_count".format(team["team_number"], p_idx)] = player['assists_count']
                tmp["team_{}_player_{}_denies_count".format(team["team_number"], p_idx)] = player['denies_count']
                tmp["team_{}_player_{}_lh_count".format(team["team_number"], p_idx)] = player['lh_count']
                tmp["team_{}_player_{}_gold".format(team["team_number"], p_idx)] = player['gold']
                tmp["team_{}_player_{}_net_worth".format(team["team_number"], p_idx)] = player['net_worth']

                for i_idx, item in enumerate(player['items']):
                    tmp["team_{}_player_{}_item_{}".format(team["team_number"], p_idx, i_idx)] = item

        for gg_idx, graph_gold in enumerate(entry['graph_data']['graph_gold']):
            tmp['graph_gold_{}'.format(gg_idx)] = graph_gold

        for i, building in enumerate(entry["buildings"]):
            tmp["building_{}".format(i)] = int(building["destroyed"])

        matches_for_pd.append(tmp)
    except Exception as e:
        MLRS_COL.delete_one({"_id": entry['_id']})
        print(e, entry['_id'])

In [17]:
df = pd.DataFrame(matches_for_pd)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Columns: 338 entries, timestamp to building_35
dtypes: int64(338)
memory usage: 34.5 KB


In [18]:
print(df.columns)

Index(['timestamp', 'game_time', 'score', 'net_worth', 'team_2_player_0_level',
       'team_2_player_0_kill_count', 'team_2_player_0_death_count',
       'team_2_player_0_assists_count', 'team_2_player_0_denies_count',
       'team_2_player_0_lh_count',
       ...
       'building_26', 'building_27', 'building_28', 'building_29',
       'building_30', 'building_31', 'building_32', 'building_33',
       'building_34', 'building_35'],
      dtype='object', length=338)


In [19]:
X = df.iloc[:, :].values
print(X)

[[1437 1097    8 ...    0    0    0]
 [1591 1250    7 ...    0    0    0]
 [1747 1406    5 ...    0    0    0]
 ...
 [2896 2555    3 ...    0    0    0]
 [2972 2631    1 ...    0    0    0]
 [3051 2711    2 ...    0    0    0]]


In [20]:
print(X[-4])

[ 2667  2327     9  3224    19     2     5    17     7   184  2535 15780
    50    34   119    40    31   108    -1    -1    13    18     7     8
    13     2   143  2315 14360   100    40   178   214   108    73    73
    -1    -1    18     6     8    18     6    42  1200  9455   180     1
   254   188    40   232  4204  1123    -1    24    11     0     7    11
   298  2122 23872    48    30   600   123   692   116    -1    -1    -1
    23     8     4    10    24   418  2630 24830   154   117   116    63
   158   123    26     5    -1    23     9     5     5    20   362   102
 22752   116   160   147    63  1808    81    -1    -1    -1    19     6
     4    16     4   222  1811 17186   263   180    36  1107   147    41
    -1    -1    -1    20     4     9    17     0   170   760 15460   152
    50   108   162   692   244    -1    -1    -1    18     4     6    17
     5   104   613 12868   232   108   180     1    36    40    -1   244
   188    21     1    10     7     4   236  4407 16

In [35]:
for pred in et_classifier.predict_proba(X):
    if pred[0] > pred[1]:
        print('dire', round(pred[0], 2))
    else:
        print('radiant', round(pred[1], 2))

radiant 0.7
radiant 0.67
radiant 0.51
radiant 0.61
radiant 0.59
radiant 0.55
radiant 0.64
radiant 0.76
radiant 0.67
radiant 0.68
dire 0.53
dire 0.61
dire 0.68


In [36]:
for pred in gb_classifier.predict_proba(X):
    if pred[0] > pred[1]:
        print('dire', round(pred[0], 2))
    else:
        print('radiant', round(pred[1], 2))

dire 0.81
dire 0.99
dire 0.97
dire 0.54
radiant 0.69
radiant 0.71
radiant 0.6
radiant 0.87
radiant 0.85
radiant 0.93
radiant 0.74
dire 0.72
dire 0.93


In [37]:
for pred in hgb_classifier.predict_proba(X):
    if pred[0] > pred[1]:
        print('dire', round(pred[0], 2))
    else:
        print('radiant', round(pred[1], 2))

radiant 0.96
radiant 0.8
radiant 0.68
radiant 0.88
radiant 0.82
radiant 0.82
radiant 0.74
radiant 0.84
radiant 0.71
radiant 0.71
dire 0.58
dire 0.74
dire 0.83


In [38]:
for pred in rf_classifier.predict_proba(X):
    if pred[0] > pred[1]:
        print('dire', round(pred[0], 2))
    else:
        print('radiant', round(pred[1], 2))

radiant 0.64
radiant 0.64
dire 0.62
radiant 0.6
radiant 0.68
radiant 0.66
radiant 0.64
radiant 0.72
radiant 0.62
radiant 0.56
dire 0.66
dire 0.8
dire 0.62
